In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

from pose_estimation import PoseData, PoseDataNPZ, PoseDataNPZTorch, COLOR_PALETTE

WORKDIR = f"{os.getcwd()}/.."
DATA_FOLDER = os.path.join(WORKDIR, "data_folder")
DATASET_NPZ_PATH = os.path.join(DATA_FOLDER, "dataset_npz")


In [2]:
dataset = None
dataset = PoseDataNPZ(DATASET_NPZ_PATH)


Presumed Preloaded NPZ Dataset: /Users/armanommid/Code/CSE/CSE275/HW2/XSwinDiffusion/../data_folder/dataset_npz


In [26]:
x = 0
for i, key in enumerate(dataset.keylist):
    id1 = dataset.meta(key)["object_ids"]
    id2 = dataset.meta(key)["objects"]

    if not np.array_equal(id1, id2):
        # print(type(id1), type(id2))
        # print(id1, id2)
        x += 1
print(x)


466
